In [ ]:
from ollama import Client
from pydantic import BaseModel

class ExampleOutputClass(BaseModel):
    response : str
    evidences : list[str]


def multiply(a : int, b : int):
    """
    Multiples two numbers together.

    Args:
        a: First number.
        b: Second number.
    
    Returns:
        The result of multiplying a and b.
    """

    return a * b

client = Client()

available_function = {"multiply" : multiply}

response = client.chat(
    model="llama3.2:latest", 
    messages=[{"role": "user", "content": "What is five times 3? Your response MUST be in JSON format."}],
    tools = [multiply],
    format = ExampleOutputClass.model_json_schema(),
    options = {"temperature" : "0"}
)

print(response["message"]["content"])

pythonic_output = ExampleOutputClass.model_validate_json(response["message"]["content"])
print(pythonic_output)

for tool in response.message.tool_calls or []:
    function_to_call = available_function.get(tool.function.name)
    if function_to_call:
        print(f"Result: {function_to_call(**tool.function.arguments)}")

# Huggingface Function Calling